In [3]:
# import modules

import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import CRS
import shapely.speedups
import folium
from folium.plugins import Search, MarkerCluster
from shapely.geometry import Polygon, MultiPolygon
import os, sys
%matplotlib inline


Let's get some data

In [4]:
# root data
root = r"D:\Gustavo\resource_center\RLIS"

# import data
school_fp =  "PLACES\schools.shp"
lib_fp = "PLACES\library.shp"
districts_fp = "CENSUS\district20.shp"

Convert into GeoDataFrame, and change the column name from `UPPECASE` to `lowercase`

In [5]:
# Shapefile to GeoDataFrame
schools = gpd.read_file(os.path.join(root, school_fp))
schools.columns = schools.columns.str.lower() # from UPPERCASE to lowercase

libraries = gpd.read_file(os.path.join(root, lib_fp))
libraries.columns = libraries.columns.str.lower()

districts = gpd.read_file(os.path.join(root, districts_fp))
districts.columns = districts.columns.str.lower()

DriverError: D:\Gustavo\resource_center\RLIS\PLACES\schools.shp: No such file or directory

In [ ]:
# check the Coordinate system of the geodataframes
print(schools.crs)
print(libraries.crs)
print(districts.crs)

In [ ]:
# re-project zipcode geodataframe:
# this is done to be able to...
schools = schools.to_crs(epsg=4326)
libraries = libraries.to_crs(epsg=4326)
districts = districts.to_crs(epsg=4326)
"""Note: re-run abouve cell to verify that the crs's were changed"""

In [ ]:
# check the Coordinate system of the geodataframes
print(schools.crs)
print(libraries.crs)
print(districts.crs)

Let's retrieve `Latitude` and `Longitude` from the geometry column in schools

In [ ]:
schools['lon']= schools.geometry.x # Add lon column to geodataframe
schools['lat']= schools.geometry.y # Add lon column to geodataframe

In [ ]:
schools.head()
# schools.columns

In [ ]:
# Check if variables are GeoDataFrames

print(['geometry column exists' for geom in schools.columns if geom == 'geometry'])
print(['geometry column exists' for geom in libraries.columns if geom == 'geometry'])
print(['geometry column exists' for geom in districts.columns if geom == 'geometry'])

In [ ]:
# Create a index column in school, libraries, zipcode, geodataframes
schools['geoid'] = schools.index.astype(str)
libraries['geoid'] = libraries.index.astype(str)
districts['geoid'] = districts.index.astype(str)

# # # select column of interest
schools_col= ['geoid', 'name', 'city', 'state', 'zipcode', 'type', 'county', 'geometry', 'lon', 'lat']
libraries_col= ['geoid', 'zipcode', 'name', 'address', 'city', 'state',  'geometry']
districts_col= ['geoid', 'pop00', 'geometry']

# # re-defining the geodataframes
schools = schools[schools_col]
libraries = libraries[libraries_col]
districts = districts[districts_col]



In [ ]:
# print the lenght of all geodataframes
print(f'lenght of Schools is: {len(schools)}')
print(f'Lenght of Libraries is: {len(libraries)}')
print(f'Lenght of districts is: {len(districts)}')

In [ ]:
# Get description of the GeoDataFrame districts
districts.describe()
# districts.shape

Based on the description of the districts gdf; we can see how far are the min and the max values for population are from top to bottom quartiles breakpoints. This indicate that are some outliers that are well outside of the mean of most of the distribution.
<br>
Let's take a closer look at it.

In [ ]:
# Create a gdf Sort by population 
dist_pop = districts.sort_values(by='pop00', ascending=False)
dist_pop.head(5).append(dist_pop.tail(5))

Let's define min and max values

In [ ]:
# Define min and max values
min_, max_ = dist_pop['pop00'].quantile([0.05, 0.95]).apply(lambda x: round(x, 2))

# Define Mean value
mean = round(dist_pop['pop00'].mean(), 2)

# print the min, max and mean values
print(f'min: {min_}\nmax: {max_}\nMean: {mean}')

Let's make a color scale<br>
For this case we are going to use a colorscale with a sequential light-to-dark color palette from [ColorBrewer](https://colorbrewer2.org/#type=sequential&scheme=Purples&n=5) website
Another great resource is [Folium ColorMap samples](https://notebook.community/ocefpaf/folium/examples/Colormaps)


In [1]:
import branca 

# Define color pallet
color_pallet = ['#f0f9e8','#ccebc5','#a8ddb5','#7bccc4','#4eb3d3', '#2b8cbe', '#08589e']

## colormap = branca.colormap.LinearColormap(
#     colors= color_pallet,
#     index= dist_pop['pop00'].quantile([0.15,0.3,0.45,0.6, 0.7, 0.8]),
#     vmin = min_,
#     vmax = max_
#     )

# Find source and samples in https://notebook.community/ocefpaf/folium/examples/Colormaps
# syntax : branca.colormap.linear.<colorscale>_<#classes>.scale(min_value, max_value)
colormap = branca.colormap.linear.YlGnBu_07.scale(min_, max_) 


# color bar tittle
colormap.caption= 'Population in Portland Oregon in 2010'

colormap

NameError: name 'min_' is not defined

__Alright__ Time to build a Map<br>
Let's start by clustering all the points

In [ ]:
# Create dataframe series from lat and lon columns
lat = schools['lat']
lon = schools['lon']

# Create dataframe series from name, city, state zipcode columns
name = schools['name']
city = schools['city']
state = schools['state']
zipcode = schools['zipcode']

locations = list(zip(lat, lon)) # list of the lat and lon from the schools gdf

naming = list(zip(name, city, state, zipcode)) # list of all name, city, state, zipcode

# Create formating for popups to define what to show in each location
popups = [f'Name: {name}<br>City: {city}<br>State: {state}<br>Zipcode: {zipcode}' for (name, city, state, zipcode) in naming]

In [ ]:
locations

__Let's build the map__

In [ ]:
%%time

# Create map instance
m = folium.Map(location=[np.mean(lat), np.mean(lon)], width="100%", height="100%", 
               zoom_start=10, min_zoom= 9, max_zoom=19, 
               tiles="OpenStreetMap")

m

In [ ]:
%%time

# Create map instance
m = folium.Map(location=[np.mean(lat), np.mean(lon)], width='100%', height='100%',
              zoom_start=2, min_zoom=9, max_zoom=19,
              tiles= 'cartodbpositron', control_scale=True
            )

# Define style
style_function=lambda x: {
    'fillColor': colormap(x['properties']['pop00']),
    'color': 'white',
    'weight': 2,
    'fillOpacity': 0.5
    }

# Add Population layer to m
pop_geo = folium.GeoJson(
    dist_pop,
    name= 'Districts',
    style_function= style_function,
    tooltip= folium.GeoJsonTooltip(
    fields=['pop00'],
    aliases=['Population: '],
    localize=True
    )
).add_to(m)

# # # # ======================================== # # # #  
# Add marker cluster layer for the schools
marker_cluster = MarkerCluster()
marker_cluster = folium.plugins.FastMarkerCluster(locations)


marker_cluster = MarkerCluster(
    locations= locations, popups= popups,
    name = 'Schools',
    overlay= True,
    control= True
)

# # Adding Schools Seach Bar:
# schools_seach = Search(
#     layer=schools,
#     geom_type='Point',
#     placeholder='Seach for schools in Oregon',
#     collapsed=True,
#     search_label='School_name'
# ).add_to(m)

# Add marker cluster
marker_cluster.add_to(m)

# Add layer control
colormap.add_to(m)
folium.LayerControl().add_to(m)

# creates web map with just the map and some interactivity
# root2 = r"C:\Users\gcolm\Documents\courses\online\online_courses\others\Geo_python_course\autoGIS_2019\lesson_5"
# m.save(os.path.join(root2, 'cluster_schools_in_the_PNW_2.html'))

# Display 
m

